<h1 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
    DeepSeek-R1 with Amazon Bedrock 
</h1>

# Prerequisites

- An AWS account with access to Amazon Bedrock
- Sufficient local storage space (at least 17GB for 8B and 135GB for 70B models)
- (Optional) An Amazon S3 bucket prepared to store the custom model
- (Optional) An AWS IAM Role with permissions for Bedrock to read from S3

## Step 1: Install required packages

In [ ]:
!pip install boto3 huggingface_hub transformers[torch] -U

## Step 2: Configure parameters
Note: When using the defaults, unique random strings will be appended to resource names to prevent naming conflicts.

In [ ]:
from botocore.config import Config

aws_session_config = Config(retries={'total_max_attempts': 10, 'mode': 'standard'})

# Default configuration values
default_region = 'us-east-1'
default_repository_id = 'deepseek-ai/DeepSeek-R1-Distill-Llama-8B'
default_s3_root_folder = '/'
default_s3_bucket_base_name = 'bedrock-imported-models'
default_import_role_base_name = 'AmazonBedrockModelImportRole'
default_import_policy_name = 'AmazonBedrockModelImportPolicy'

# Collect required parameters from user
# Allow user to specify custom Hugging Face model repository
repository_id = input(f"Enter Hugging Face repository ID ['{default_repository_id}']: ") or default_repository_id

# Allow user to specify AWS region for deployment
aws_region = input(f"Enter the AWS region: ['us-east-1']: ") or default_region

# Get IAM role name for model import permissions
# If left empty, a new role will be created automatically
import_role_name = input("Enter the IAM role name for the model import [Leave empty to create a new role]") or None

# Get S3 storage configuration
# If left is empty, a new bucket will be created
s3_bucket_name = input('Enter the S3 bucket name [Leave empty to create a new bucket]') or None
s3_root_folder = input(f"Enter the S3 root prefix ['{default_s3_root_folder}']") or default_s3_root_folder

# Display final configuration settings
print('Configuration:')
print(f"- HF Repository ID: {repository_id}")
print(f"- Import role ARN: {import_role_name or 'Create a new IAM role'}")
print(f"- S3 bucket: {s3_bucket_name or 'Create a new S3 bucket'}")
print(f"- S3 root folder: {s3_root_folder}")


## Step 3: Create new IAM Role and S3 Bucket if required

In [ ]:
import boto3
import json
import random
import string

from botocore.exceptions import ClientError

# Create a random resource name postfix
postfix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))

def get_aws_account_id():
    sts_client = boto3.client('sts')
    return sts_client.get_caller_identity()['Account']

def get_or_create_role(role_name):
    iam_client = boto3.client('iam')
    
    if not role_name:
        print('Creating new IAM role...')
        
        account_id = get_aws_account_id()
        role_name = f"{default_import_role_base_name}-{postfix}"
       
        trust_policy = {
            "Version": "2012-10-17",
            "Statement": [{
                "Effect": "Allow",
                "Principal": { "Service": "bedrock.amazonaws.com" },
                "Action": "sts:AssumeRole",
                "Condition": {
                    "StringEquals": { "aws:SourceAccount": account_id },
                    "ArnEquals": {
                        "aws:SourceArn": f"arn:aws:bedrock:{aws_region}:{account_id}:model-import-job/*"
                    }
                }
            }]
        }
        
        inline_policy = {
            "Version": "2012-10-17",
            "Statement": [{
                    "Effect": "Allow",
                    "Action": [
                        "s3:GetObject",
                        "s3:ListBucket"
                    ],
                    "Resource": [
                        s3_bucket_arn,
                        f"{s3_bucket_arn}/*"
                    ],
                    "Condition": {
                        "StringEquals": {
                            "aws:ResourceAccount": account_id
                        }
                    }
                }
            ]
        }
        
        try:
            role = iam_client.create_role(
                RoleName=role_name,
                AssumeRolePolicyDocument=json.dumps(trust_policy)
            )

            iam_client.put_role_policy(
                RoleName=role_name,
                PolicyName=f"{default_import_policy_name}",
                PolicyDocument=json.dumps(inline_policy)
            )
            
            print(f"Successfully created IAM role: {role_name}")
        except ClientError as e:
            print(f"Error creating IAM role: {e}")
            exit(1)
    else:
        print(f"Checking IAM role: {role_name}")
        try:
            role = iam_client.get_role(RoleName=role_name)
            print('Found existing role.')
        except ClientError as e:
            print(f"Error retrieving S3 bucket: {e}")
            exit(1)
            
    return role["Role"]

def get_or_create_bucket(bucket_name):
    s3_client = boto3.client('s3', region_name=aws_region)
    
    if not bucket_name:
        print(f"Creating new S3 bucket...")
        
        bucket_name = f"{default_s3_bucket_base_name}-{postfix}"
        
        try:
            s3_client.create_bucket(Bucket=bucket_name)
            
            # Wait until bucket exists
            waiter = s3_client.get_waiter('bucket_exists')
            waiter.wait(
                Bucket=bucket_name,
                WaiterConfig={
                    'Delay': 5,
                    'MaxAttempts': 20
                }
            )
            
            print(f"Successfully created S3 bucket: {bucket_name}")
            return bucket_name
            
        except ClientError as e:
            print(f"Error creating S3 bucket: {e}")
            exit(1)
    else:
        print(f"Checking S3 bucket: {bucket_name}")
        try:
            bucket = s3_client.head_bucket(Bucket=bucket_name)
            print('Found existing bucket')
        except ClientError as e:
            print(f"Error retrieving IAM role: {e}")
            exit(1)
            
s3_bucket_arn = f"arn:aws:s3:::{s3_bucket_name}"
import_role = get_or_create_role(import_role_name)
import_role_arn = import_role["Arn"]

s3_bucket = get_or_create_bucket(s3_bucket_name)
s3_bucket_arn = f"arn:aws:s3:::{s3_bucket_name}"

---

# Download and deploy the model
## Step 1: Download the weights from Hugging Face

In [ ]:
from huggingface_hub import snapshot_download

local_dir = snapshot_download(repository_id)
print(f"Model downloaded to: {local_dir}")

## Step 2: Upload the weights to Amazon S3

In [ ]:
import os

s3 = boto3.client('s3')

s3_folder = repository_id if s3_root_folder == '/' else f'{s3_root_folder}/{repository_id}'
s3_folder_uri = f"s3://{s3_bucket_name}/{s3_folder}"

def file_exists_in_s3(bucket_name, s3_key):
    return s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_key)['KeyCount'] > 0

def upload_to_s3():
    for root, _, files in os.walk(local_dir):
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, local_dir)
            s3_key = f"{s3_folder}/{relative_path}"
            
            if file_exists_in_s3(s3_bucket_name, s3_key):
                print(f"Skipping existing file: s3://{s3_bucket_name}/{s3_key}")
                continue
                
            print(f"Uploading: {file_path} to s3://{s3_bucket_name}/{s3_key}")
            s3.upload_file(file_path, s3_bucket_name, s3_key)

print('Uploading model files to S3...')

upload_to_s3()

print(f"Successfully uploaded model files to {s3_folder_uri}")

## Step 3: Deploy the model to Amazon Bedrock
### 3.1: Start a model import job 

In [ ]:
import datetime

bedrock = boto3.client('bedrock', region_name=aws_region)
model_name = repository_id.split('/')[-1].replace('.', '-').replace('_', '-')

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

job_name = f'{model_name}-{timestamp}'

print(f"Starting model import job: {job_name}")

# Create the model import job
response = bedrock.create_model_import_job(
    jobName=job_name,
    importedModelName=model_name,
    roleArn=import_role_arn,
    modelDataSource={'s3DataSource': {'s3Uri': s3_folder_uri}}
)

print(f"Model import job started")

job_arn = response['jobArn']


### 3.2: Monitor the import job status

In [ ]:
import time

print(f"Checking status of import job: {job_name}")
while True:
    response = bedrock.get_model_import_job(jobIdentifier=job_arn)
    status = response['status']
    if status == 'Failed':
        print('Model import failed!')
        
        failure_message = response['failureMessage']
        print(f"Reason: {failure_message}")
        break
    elif status == 'Completed':
        print('Model import complete.')
        
        model_id = response['importedModelArn']
        print(f"Imported model ID: {model_id}")
        break
    else:
        print('Importing...')

    time.sleep(60)  # Check every 60 seconds
    
model_arn = response['importedModelArn']


### 3.3: If necessary, wait some more time to make sure the model has been initialized

In [ ]:
# Wait for 5 minutes for model initialization 
print('Waiting 5 minutes for model initialization...')

for i in range(5, 0, -1):
    print(f'{i} minute{"s" if i > 1 else ""}...')
    time.sleep(60)

# Test the model

## Step 1: Prepare the prompt 

In [ ]:
# Define the message you want to send to the model
user_message = "Explain the concept of 'rubber duck debugging' in a single paragraph."

# Apply the `user` role and embed it in a list of messages
messages = [
    {'role': 'user','content': user_message}
]

# For optimal results, apply the specific model's chat template
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(repository_id)

formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(f"Formatted prompt: \n{formatted_prompt}")

## Step 2: Invoke the model
 
### Option 2.1: Using the `InvokeModelWithResponseStream` API
This allows you to retrieve and process the response in real-time as chunks.

**Pros:** Lower latency to first displayed content; provides visual feedback during generation; reduced memory overhead for large responses

**Cons:** More complex implementation; requires handling partial/incomplete responses; need to manage stream state and error handling

In [ ]:
# Define a function to invoke the model and process the response stream in real-time
def invoke_model_with_response_stream(prompt, region_name=aws_region, max_tokens=4096):
    global model_arn
    global aws_session_config
        
    session = boto3.session.Session()
    br_runtime = session.client(
        service_name='bedrock-runtime',
        region_name=region_name,
        config=aws_session_config
    )
    
    payload = {
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": 0.7,
        "top_p": 0.9
    }
    streaming_response = br_runtime.invoke_model_with_response_stream(
        modelId=model_arn,
        body=json.dumps(payload)
    )
    
    final_response = ""
    for event in streaming_response["body"]:
        chunk = json.loads(event['chunk']['bytes'])
        if "generation" in chunk:
            text = chunk["generation"]
            final_response += text 
            print(text, end="", flush=True)
            
    return final_response

# Invoke the model with the formatted prompt and observe its output in real-time
response = invoke_model_with_response_stream(formatted_prompt)

### Option 2.2: Using the `InvokeModel` API
This provides a single, complete response after model processing is finished.

**Pros:** Simpler implementation; response arrives fully formatted; easier to use for downstream processing

**Cons:** Longer perceived wait time; no visual feedback during processing; entire response must be held in memory at once

In [ ]:
# Define a function to invoke the model and return the complete response
def invoke_model(model, message, region_name=aws_region, max_tokens=4096):
    global model_arn
    global aws_session_config
        
    session = boto3.session.Session()
    br_runtime = session.client(
        service_name='bedrock-runtime',
        region_name=region_name,
        config=aws_session_config
    )
    
    payload = {
        "prompt": formatted_prompt,
        "max_tokens": max_tokens,
        "temperature": 0.7,
        "top_p": 0.9
    }
        
    try:
        complete_response = br_runtime.invoke_model(
            modelId=model, 
            body=json.dumps(payload) 
        )
        result = json.loads(complete_response["body"].read().decode("utf-8"))
    except Exception as e:
        print(e)
        print(e.__repr__())

    return result["generation"]

response = invoke_model(model=model_arn, message=user_message)

## Step 3: Display the formatted final response

In [ ]:
from IPython import display

def try_convert_to_markdown(raw_response):
    # When reasoning section (<think>...</think>) is present, convert the response into markdown
    if '<think>' in raw_response and '</think>' in raw_response:
        # Extract the reasoning content between <think> tags
        think_start = raw_response.find('<think>') + len('<think>')
        think_end = raw_response.find('</think>')
        reasoning_section = raw_response[think_start:think_end]

        # Get the actual response that follows the reasoning section
        response_section = raw_response[raw_response.find('</think>') + len('</think>'):]

        # Format the output with markdown headers to visually separate the reasoning from the final response
        return (f"\n**Reasoning:**\n" \
                f"\n{reasoning_section.strip()}\n" \
                f"\n**Response:**\n" \
                f"\n{response_section.strip()}\n")

    else:
        # When no reasoning section is present, return the raw generation without any special formatting
        return raw_response

# Convert the model output to markdown format and display it
converted_response_text = try_convert_to_markdown(response)
display.Markdown(converted_response_text)